In [ ]:
%pip install datasets
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 182 kB 86.7 MB/s 
     |████████████████████████████████| 115 kB 9.1 MB/s 
     |████████████████████████████████| 212 kB 84.7 MB/s 
     |████████████████████████████████| 127 kB 89.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 5.5 MB 27.8 MB/s 
     |████████████████████████████████| 1.3 MB 67.1 MB/s 
     |████████████████████████████████| 7.6 MB 78.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=040b1502c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from datetime import datetime
import re
import math
from itertools import cycle
from sentence_transformers.datasets import ParallelSentencesDataset
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses
import logging
data_path = '/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/dataset/'
model_path = '/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/models/'
data_csv=pd.read_csv(data_path+'translation.csv')
data_csv = data_csv.dropna()

In [ ]:
code_path = '/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/'

In [ ]:
import gc
#del variables
gc.collect()

166

In [ ]:
gc.collect()

219

In [ ]:
import torch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
out = {}
a=re.compile("<.*?>")
for _, string in data_csv.iterrows():
    string1 = re.sub("[\(\[].*?[\)\]]", "", string['source']).rstrip('\t\n').lstrip()
    string2 = re.sub("[\(\[].*?[\)\]]", "", string['translation']).rstrip('\t\n').lstrip()
    string1 = a.sub('',string1)
    string2 = a.sub('',string2)
    string1 = string1.replace('*', '')
    string2 = string2.replace('*', '')
    # if string1 == '':
    #   print ("yes")
    if string1 == '' or string2 == '':
      print ("yes")
      continue
    string1 = string1.replace('"', '')
    string2 = string2.replace('"', '')
    string1 = string1.replace('«', '')
    string2 = string2.replace('«', '')
    string1 = string1.replace('»', '')
    string2 = string2.replace('»', '')

    string1 = " ".join(string1.split())
    string2 = " ".join(string2.split())
    if string['lang'] not in out:
        out[f"{string['lang']}"] = [string1,string2]
    else:
        out[f"{string['lang']}"].extend([string1,string2])

In [ ]:
for key in out:
    i=0
    with open(data_path+f"{key}.tsv","w") as f:
      with open(data_path+f"{key}-ru.txt", "w") as Ru:
        with open(data_path+f"ru-{key}.txt", "w") as nonRu:
          licycle = cycle(out[key])
          # Prime the pump
          while i<len(out[key])/2:
            i += 1
            thiselem = next(licycle)
            nextelem = next(licycle)
            if thiselem.isspace() or nextelem.isspace():
              continue
            f.write(thiselem+"\t"+nextelem+"\n")
            Ru.write(nextelem+"\n")
            nonRu.write(thiselem+"\n")

In [ ]:
#filenames = [data_path+"evn.tsv", data_path+"evn_corpus.tsv", data_path+"inel.tsv"]
#filenames = [data_path+"evn.tsv", data_path+"inel.tsv"]
filenames = [data_path+"evn.tsv", data_path+"evn_corpus.tsv"]
with open(data_path+'new_evn.tsv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
ru = pd.read_csv(data_path+"kjh-ru.txt", sep="\n", header=None)

In [ ]:
kjh = pd.read_csv(data_path+"ru-kjh.txt", sep="\n", header=None)

In [ ]:
kjh_train, kjh_validate, kjh_test = \
              np.split(kjh.sample(frac=1, random_state=42), 
                       [int(.97*len(kjh)), int(.985*len(kjh))])

In [ ]:
ru_train, ru_validate, ru_test = \
              np.split(ru.sample(frac=1, random_state=42), 
                       [int(.97*len(ru)), int(.985*len(ru))])

In [ ]:
kjh_train.to_csv(code_path+"afrobart/ru-kjh/train.kjh",index=False, header=False, sep="\n", quoting=None)
kjh_test.to_csv(code_path+"afrobart/ru-kjh/test.kjh",index=False, header=False, sep="\n", quoting=None)
kjh_validate.to_csv(code_path+"afrobart/ru-kjh/validate.kjh",index=False, header=False, sep="\n", quoting=None)

In [ ]:
ru_train.to_csv(code_path+"afrobart/ru-kjh/train.ru",index=False, header=False, sep="\n", quoting=None)
ru_test.to_csv(code_path+"afrobart/ru-kjh/test.ru",index=False, header=False, sep="\n", quoting=None)
ru_validate.to_csv(code_path+"afrobart/ru-kjh/validate.ru",index=False, header=False, sep="\n", quoting=None)

In [ ]:
ru_train

,0
4708,тунук знаешь ведь что такое?
691,"сейчас, когда уже стала стареть, нужным это ок..."
65,с детьми…
3424,есть.
4680,а муж твой где работает?
...,...
4659,"ох, потом все делаем, телят загоняем, комнаты ..."
1853,какие бисеры?
3763,"у нас в верх-аскизе толмашовы, сунчугашевы, уг..."
3304,"одежду, которую она в молодости носила, меняла..."


In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [ ]:
teacher_model_name = "DeepPavlov/rubert-base-cased-conversational"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
teacher_model_name = "sentence-transformers/all-mpnet-base-v2"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
# teacher_model_name = "sberbank-ai/sbert_large_nlu_ru"
# student_model_name = "sentence-transformers/LaBSE"

In [ ]:
teacher_model_name

'DeepPavlov/rubert-base-cased-conversational'

In [ ]:
#0.30
teacher_model_name = "DeepPavlov/rubert-base-cased-conversational"
student_model_name = "DeepPavlov/distilrubert-small-cased-conversational"

In [ ]:
teacher_model_name = "sberbank-ai/sbert_large_nlu_ru"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
#teacher_model_name = 'paraphrase-distilroberta-base-v2'   #Our monolingual teacher model, we want to convert to multiple languages
#student_model_name = 'xlm-roberta-base'       #Multilingual base model we use to imitate the teacher model
student_model_name = "bert-base-multilingual-cased"

In [ ]:
#teacher_model_name = model_path+'sentence_ru_cased_L-12_H-768_A-12_pt/'
teacher_model_name = model_path+"lorelei_29_langs/rus/step_500k"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
#0.117
teacher_model_name = 'all-distilroberta-v1'
student_model_name = "DeepPavlov/distilrubert-small-cased-conversational"

In [ ]:
# teacher_model_name = "microsoft/MiniLM-L12-H384-uncased"
# student_model_name = "Unbabel/xlm-roberta-comet-small"

In [ ]:
#translation=0.11
teacher_model_name = 'all-distilroberta-v1'
student_model_name = "bert-base-multilingual-cased"

In [ ]:
teacher_model_name = 'all-distilroberta-v1'
student_model_name = model_path+'sentence_ru_cased_L-12_H-768_A-12_pt/'

In [ ]:
teacher_model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
student_model_name = "bert-base-multilingual-cased"

In [ ]:
teacher_model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
teacher_model_name = "DeepPavlov/rubert-base-cased-sentence"

In [ ]:
teacher_model_name = "sberbank-ai/sbert_large_nlu_ru"
student_model_name = "xlm-roberta-large"

In [ ]:
teacher_model_name = "sberbank-ai/ruBert-base"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
teacher_model_name = "mrm8488/setfit-distiluse-base-multilingual-cased-v2-finetuned-amazon-reviews-multi-binary"
student_model_name = "sentence-transformers/LaBSE"

In [ ]:
logger.info("Load teacher model")
teacher_model = SentenceTransformer(teacher_model_name)
# logger.info("Create student model from scratch")
# word_embedding_model = models.Transformer(student_model_name, max_seq_length=max_seq_length)
# # Apply mean pooling to get one fixed sized sentence vector
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [ ]:
max_seq_length = 128
train_batch_size = 64
num_epochs = 10
inference_batch_size = 64
num_warmup_steps = 1000             #Warumup steps
num_evaluation_steps = 100          #Evaluate performance after every xxxx steps
dev_sentences = 500
max_sentences_per_language = 500000 #Maximum number of  parallel sentences for training
train_max_sentence_length = 250     #Maximum length (characters) for parallel training sentences

In [ ]:
logger.info("Create student model from scratch")
word_embedding_model = models.Transformer(student_model_name, max_seq_length=max_seq_length)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
student_model = SentenceTransformer(student_model_name)

In [ ]:
student_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
teacher_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("stsb_multi_mt", name="ru", split="dev")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/ru/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


In [ ]:
sts_data = {'sentences1': [], 'sentences2': [], 'scores': []}

In [ ]:
for line in dataset:
    sent1, sent2, score = line['sentence1'], line['sentence2'], line['similarity_score']
    score = float(score)
    sts_data['sentences1'].append(sent1)
    sts_data['sentences2'].append(sent2)
    sts_data['scores'].append(score)

In [ ]:
devFile = data_path+"train.tsv"
trainFile = data_path+"dev.tsv"

In [ ]:
source_languages = set(['ru'])                      # Our teacher model accepts English (en) sentences
target_languages = set(['bak', 'evn', 'kjh', 'mhr', 'xas'])

In [ ]:
###### Load dev sets ######

evaluators = []


###### Load test sets ######

# test_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
# test_reader.load_data('dataset/kjh-test.txt')


fTrain = open(devFile, 'wt', encoding='utf8')
fDev = open(trainFile, 'wt', encoding='utf8')

with open(data_path+'new_evn.tsv', 'rt', encoding='utf8') as fIn:
#with open(data_path+'kjh.tsv', 'rt', encoding='utf8') as fIn:
    #reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    devCount = 0
    for line in fIn:
        #print (line)
        src_text = line.split('\t')[1].strip()
        trg_text = line.split('\t')[0].strip()
                
        #print(src_text, trg_text)
        if src_text != "" and trg_text != "":
            if devCount < dev_sentences:
                devCount += 1
                fOut = fDev
            else:
                fOut = fTrain

            fOut.write("{}\t{}\n".format(src_text, trg_text))


fTrain.close()
fDev.close()


train_data = ParallelSentencesDataset(student_model=student_model, teacher_model=teacher_model, batch_size=inference_batch_size, use_embedding_cache=True)
train_data.load_data(trainFile, max_sentences=max_sentences_per_language, max_sentence_length=train_max_sentence_length)

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
#num_warmup_steps = math.ceil(len(train_dataloader) * 0.1 * num_epochs)
train_loss = losses.MSELoss(model=student_model)


logger.info("Create evaluator for dev")
src_sentences = []
trg_sentences = []
with open(devFile, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        splits = line.strip().split('\t')
        if splits[0] != "" and splits[1] != "":
            src_sentences.append(splits[0])
            trg_sentences.append(splits[1])

dev_mse = evaluation.MSEEvaluator(src_sentences, trg_sentences, name='dev', teacher_model=teacher_model, batch_size=inference_batch_size)
evaluators.append(dev_mse)

dev_trans_acc = evaluation.TranslationEvaluator(src_sentences, trg_sentences, name='trans', batch_size=inference_batch_size)
evaluators.append(dev_trans_acc)

test_evaluator = evaluation.EmbeddingSimilarityEvaluator(sts_data['sentences1'], sts_data['sentences2'], sts_data['scores'], batch_size=inference_batch_size, name='test', show_progress_bar=False)
evaluators.append(test_evaluator)

output_path = data_path+"output/make-multilingual-"+"-".join(sorted(list(source_languages))+sorted(list(target_languages)))+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

###### Train model ######

student_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: np.mean(scores)),
          epochs=num_epochs,
          warmup_steps=num_warmup_steps,
          evaluation_steps=num_evaluation_steps,
          output_path=output_path,
          save_best_model=True,
          use_amp = True,
          optimizer_params= {'lr': 1e-5, 'eps': 1e-6}
          )

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  labels = torch.tensor(labels)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([64, 1024])) that is different to the input size (torch.Size([64, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [ ]:
output_path = data_path+"output/make-multilingual-"+"-".join(sorted(list(source_languages))+sorted(list(target_languages)))+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
student_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: np.mean(scores)),
          epochs=30,
          warmup_steps=num_warmup_steps,
          evaluation_steps=num_evaluation_steps,
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 1e-5, 'eps': 1e-6}
          )

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
evaluators